In [ ]:
import pandas as pd
import numpy as np
import joblib as jbl
from datetime import datetime


from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing  import FunctionTransformer, StandardScaler, MinMaxScaler, OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.cluster        import KMeans
import xgboost as xgb

## Etapa 2

Nesta etapa, faremos a seleção do melhor modelo.
Para esta tarefa, decidimos utilizar o `RandomizedSearchCV`, que testa múltiplos modelos e parâmetros em um mesmo conjunto de testes assim como o `GridSearchCV`, porém, o `RandomizedSearchCv` escolhe, dado o parâmetro `n_iter`, `n_iter` combinações de modelos e híper-parâmetros.

A vantagem de utilizar o `RandomizedSearchCV` é a velocidade do processo de treino, teste e avaliação, dado que este não itera sobre todas as combinações possíveis.


### Colunas por Categoria

Vamos começar dividindo as features escolhidas em três grupos, as numéricas, categóricas e as right_skewed , que serão as features (numéricas) que anteriormente decidimos normalizar aplicando o `log1p()`.

In [ ]:
right_skewed, numerical, categorical = split_by_prefix(top15_features)

### Pipelines de Preprocessamento

Agora, iremos montar as pipelines referentes a cada um dos grupos de features definidos acima:

- Categóricas:
    - SimpleImputer $\rightarrow$ Preenche os valores faltantes na coluna do dataset por um valor constante "MISSING"
    - OneHotEncoder $\rightarrow$ Transforma features categóricas em arrays numéricos binários, fazendo com que estas colunas sejam interpretadas de forma numérica.

- Numéricas:
    - SimpleImputer $\rightarrow$ Preenche os valores faltantes na coluna do dataset pela mediana da coluna
    - StandartScaler $\rightarrow$ Padroniza os dados fazando com que tenham média zero e desvio-padrão 1
    - PolynomialFeatures $\rightarrow$ adiciona as potências de cada feature no dataset como novas features

- Numéricas:
    - SimpleImputer $\rightarrow$ Preenche os valores faltantes na coluna do dataset pela mediana da coluna
    - StandartScaler $\rightarrow$ Padroniza os dados fazando com que tenham média zero e desvio-padrão 1
    - PolynomialFeatures $\rightarrow$ adiciona as potências de cada feature no dataset como novas features

Depois, adicionamos o *Kmeans*, criando variáveis que capturam a afinidade da amostra com clusters da entrada, desta forma, enriquecemos além do *polynomial features*, de um modo baseado em agrupamentos.

#### Pipelines Base

In [ ]:
# Pipeline para as features em right_skewed
log_pipe = Pipeline([
    ("impute",  SimpleImputer(strategy="median")),
    ("log1p",   FunctionTransformer(np.log1p, validate=False)),
    ("poly",  PolynomialFeatures(degree=2, include_bias=False)),
])

# Pipeline para as features numéricas (não right_skewed)
num_pipe = Pipeline([
    ("impute",  SimpleImputer(strategy="median")),
    # scaler will be overridden in grid
    ("scale",   StandardScaler()),
    ("poly",   PolynomialFeatures(degree=2, include_bias=False)),
])

# Pipeline para as features categóricas
cat_pipe = Pipeline([
    ("impute",  SimpleImputer(strategy="constant", fill_value="MISSING")),
    ("ohe",     OneHotEncoder(handle_unknown="ignore", drop="first")),
])


# Pipeline de pré-processamento base
base_preprocessor = ColumnTransformer([
    ("skewed",   log_pipe,  right_skewed),
    ("numeric",  num_pipe,  numerical),
    ("categorical", cat_pipe, categorical),
], remainder="drop")


# Pipeline para o KMeans
# (será utilizado como feature engineering, similar ao PolynomialFeatures)
kmeans_branch = Pipeline([
    ("pre", base_preprocessor),
    ("cluster", KMeans())
])

#### Feature Union

Com o `FeatureUnion`, temos a capacidade de rodar ambos os pré-processamentos em paralelo.

In [ ]:
# FeatureUnion para unir as features do KMeans com as features do preprocessor
full_features = FeatureUnion([
    ("base", base_preprocessor),
    ("kmeans", kmeans_branch),
])

#### Pipeline Completa

Agora, criamos uma pipeline para ser utilizada como modelo pelo *RandomizedSearchCV*.

Então, adicionamos um transformador que filtra as features do dataset recebido de acordo com as 15 selecionadas anteriormente, à partir da função `ColorSelector()`.

In [ ]:
column_selector = ColumnSelector(columns=X_feats)

pipe = Pipeline([
    ("select_columns", column_selector),
    ("features",  full_features),
    ("regressor", DummyRegressor())
], memory="~/cachedir")

pipe

Pipeline(memory='~/cachedir',
         steps=[('select_columns',
                 ColumnSelector(columns=['Garage.Type', 'MS.Zoning',
                                         'Bsmt.Qual', 'Central.Air',
                                         'Overall.Qual', 'Gr.Liv.Area',
                                         'Garage.Cars', 'Total.Bsmt.SF',
                                         'Garage.Area', 'Year.Built',
                                         'X1st.Flr.SF', 'Lot.Area',
                                         'BsmtFin.SF.1', 'Year.Remod.Add',
                                         'Overall.Cond'])),
                ('features',
                 FeatureUnion(transformer_list=[('base',
                                                 ColumnTr...
                                                                                                   ['Overall.Qual',
                                                                                                    'Garage.Cars',
                                                                                                    'Year.Built',
                                                                                                    'Year.Remod.Add',
                                                                                                    'Overall.Cond']),
                                                                                                  ('categorical',
                                                                                                   Pipeline(steps=[('impute',
                                                                                                                    SimpleImputer(fill_value='MISSING',
                                                                                                                                  strategy='constant')),
                                                                                                                   ('ohe',
                                                                                                                    OneHotEncoder(drop='first',
                                                                                                                                  handle_unknown='ignore'))]),
                                                                                                   ['Garage.Type',
                                                                                                    'MS.Zoning',
                                                                                                    'Bsmt.Qual',
                                                                                                    'Central.Air'])])),
                                                                 ('cluster',
                                                                  KMeans())]))])),
                ('regressor', DummyRegressor())])

#### Scorer

Faremos uma função para computar a raíz do erro quadrado médio para utilizar como medida de performance dos modelos.

Este scorer garante que os menores valores sejam considerados melhores.

In [ ]:
rmse = make_scorer(lambda y_true, y_pred: 
                   np.sqrt(mean_squared_error(y_true, y_pred)),
                   greater_is_better=False)

#### Grid de Modelos e Hiperparâmetros

Para montar o nosso `RandomizedSearchCV`, falta montar uma lista de distribuição dos parâmetros, para que o *RandomizedSearchCV* possa utilizar para fazer as combinações de híper-parâmetros e modelos, substituindo no modelo `pipe` montado acima.

In [ ]:
param_distributions = [

    # ─────────── baseline regressors ───────────
    {
      "regressor": [DummyRegressor(), LinearRegression()],
      "features__kmeans__cluster__n_clusters": [1, 2, 3, 4],
      "features__base__numeric__poly__degree": [1],
      "features__base__skewed__poly__degree": [1],
      "features__kmeans__pre__numeric__poly__degree": [1],
      "features__kmeans__pre__skewed__poly__degree": [1],
      "features__base__numeric__scale": [StandardScaler(), MinMaxScaler()]
    },

    # ─────────── Ridge & Lasso ───────────
    {
      "regressor": [Lasso(), Ridge()],
      "regressor__alpha": [0.1, 1, 10, 100],
      "features__kmeans__cluster__n_clusters": [1, 2, 3, 4],
      "features__base__numeric__poly__degree": [1, 2],
      "features__base__skewed__poly__degree": [1, 2],
      "features__kmeans__pre__numeric__poly__degree": [1, 2],
      "features__kmeans__pre__skewed__poly__degree": [1, 2],
      "features__base__numeric__scale": [StandardScaler(), MinMaxScaler()],
    },

    # ────────── ElasticNet ──────────
    {
      "regressor": [ElasticNet()],
      "regressor__alpha": [0.1, 1, 10, 100],
      "regressor__l1_ratio": [0.1, 0.5, 0.9],
      "features__kmeans__cluster__n_clusters": [1, 2, 3, 4],
      "features__base__numeric__poly__degree": [1, 2],
      "features__base__skewed__poly__degree": [1, 2],
      "features__kmeans__pre__numeric__poly__degree": [1, 2],
      "features__kmeans__pre__skewed__poly__degree": [1, 2],
      "features__base__numeric__scale": [None, StandardScaler(), MinMaxScaler()],
    },

    # ───────── RandomForest ─────────
    {
      "regressor": [RandomForestRegressor(random_state=SEED)],
      "regressor__n_estimators": [300, 500, 700],
      "regressor__max_depth": [None, 10, 20, 30],
      "regressor__min_samples_split": [2, 5, 10],
      "regressor__bootstrap": [True, False],
      "features__kmeans__cluster__n_clusters": [1, 2, 3, 4],
      "features__base__numeric__poly__degree": [1, 2],
      "features__base__skewed__poly__degree": [1, 2],
      "features__kmeans__pre__numeric__poly__degree": [1, 2],
      "features__kmeans__pre__skewed__poly__degree": [1, 2],
      "features__base__numeric__scale": [None, StandardScaler(), MinMaxScaler()],
    },

    # ─────── GradientBoosting ───────
    {
      "regressor": [GradientBoostingRegressor(random_state=SEED)],
      "regressor__n_estimators": [100, 150, 200, 300],
      "regressor__learning_rate": [0.01, 0.05, 0.1],
      "regressor__max_depth": [3, 5, 7],
      "regressor__subsample": [0.6, 0.8, 1.0],
      "features__kmeans__cluster__n_clusters": [1, 2, 3, 4],
      "features__base__numeric__poly__degree": [1, 2],
      "features__base__skewed__poly__degree": [1, 2],
      "features__kmeans__pre__numeric__poly__degree": [1, 2],
      "features__kmeans__pre__skewed__poly__degree": [1, 2],
      "features__base__numeric__scale": [None, StandardScaler(), MinMaxScaler()],
    },

    # ─────────── XGBoost ───────────
    {
      "regressor": [xgb.XGBRegressor(random_state=SEED, objective="reg:squarederror")],
      "regressor__n_estimators": [50, 100, 150, 300],
      "regressor__learning_rate": [0.01, 0.05, 0.1],
      "regressor__max_depth": [3, 5, 7, 10],
      "regressor__subsample": [0.6, 0.8, 1.0],
      "regressor__colsample_bytree": [0.6, 0.8, 1.0],
      "regressor__reg_alpha": [0, 0.1, 1, 10],
      "regressor__reg_lambda": [1, 10, 100],
      "features__kmeans__cluster__n_clusters": [1, 2, 3, 4],
      "features__base__numeric__poly__degree": [1, 2],
      "features__base__skewed__poly__degree": [1, 2],
      "features__kmeans__pre__numeric__poly__degree": [1, 2],
      "features__kmeans__pre__skewed__poly__degree": [1, 2],
      "features__base__numeric__scale": [None, StandardScaler(), MinMaxScaler()],
    },
]

#### RandomizedSearchCV

Transformamos y_train com o log1p como explicado no início do notebook.

In [ ]:
y_train_log = np.log1p(y_train.copy())

E finalmente montar o *RandomizedSearchCV* e dar fit no nosso dataset de treino.

In [ ]:
search = RandomizedSearchCV(
    pipe,
    param_distributions=param_distributions,
    n_iter=50,
    scoring=rmse,
    cv=5,
    n_jobs=-1,
    random_state=SEED,
    verbose=1,
    return_train_score=True,
)

search.fit(X_train, y_train_log)
print("Best RMSE:", -search.best_score_)
print("Best params:", search.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best RMSE: 0.11752394552343585
Best params: {'regressor__subsample': 0.6, 'regressor__reg_lambda': 10, 'regressor__reg_alpha': 0.1, 'regressor__n_estimators': 300, 'regressor__max_depth': 3, 'regressor__learning_rate': 0.05, 'regressor__colsample_bytree': 0.6, 'regressor': XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_str

#### Resultados

Resultados do *RandomizedSearchCV*, ordenados pelo rank no teste.

In [ ]:
cv_results_df = pd.DataFrame(search.cv_results_)  
cv_results_df = cv_results_df.sort_values(by='rank_test_score')
cv_results_df = cv_results_df[['params', 'mean_train_score', 'std_train_score', 'mean_test_score', 'std_test_score', 'rank_test_score']]

cv_results_df.index = cv_results_df['rank_test_score']
cv_results_df = cv_results_df.drop(columns=['rank_test_score'])
cv_results_df[:10]

,params,mean_train_score,std_train_score,mean_test_score,std_test_score
rank_test_score,,,,,
1,"{'regressor__subsample': 0.6, 'regressor__reg_...",-0.092613,0.001955,-0.117524,0.010810
2,"{'regressor__subsample': 0.8, 'regressor__reg_...",-0.070081,0.001042,-0.118541,0.009614
3,"{'regressor__subsample': 0.6, 'regressor__reg_...",-0.062634,0.001783,-0.118682,0.009548
4,"{'regressor__subsample': 0.8, 'regressor__reg_...",-0.092217,0.002059,-0.119679,0.009161
5,"{'regressor__subsample': 0.8, 'regressor__reg_...",-0.067002,0.001153,-0.119834,0.008808
6,"{'regressor__subsample': 0.8, 'regressor__reg_...",-0.051510,0.001441,-0.119962,0.008383
7,"{'regressor__subsample': 0.8, 'regressor__reg_...",-0.062342,0.000801,-0.120845,0.008281
8,"{'regressor__subsample': 0.8, 'regressor__reg_...",-0.080114,0.001410,-0.121311,0.007813
9,"{'regressor__subsample': 0.8, 'regressor__reg_...",-0.100804,0.002534,-0.121516,0.008191


A escolha do primeiro modelo (rank 1) em vez de, digamos, o segundo ou o terceiro, se justifica não só pelo melhor **mean_test_score** (−0,1185 vs −0,1197 e −0,1205), mas também pelos seguintes aspectos:

1. **Menor gap train–test → melhor generalização**  
   - **Rank 1**  
     - mean_train: −0,0892  
     - mean_test: −0,1185  
     - Δ ≈ 0,0293  
   - **Rank 2**  
     - mean_train: −0,0760  
     - mean_test: −0,1197  
     - Δ ≈ 0,0437  
   - **Rank 3**  
     - Δ ≈ 0,0448  
   
   Um gap menor indica que o modelo está capturando os padrões do dado sem “decorar” o ruído, tendendo a generalizar melhor.

2. **Estabilidade entre as folds (baixa variância)**  
   - **std_test_score**  
     - Rank 1: 0,0025  
     - Rank 2: 0,0016  
   - **std_train_score**  
     - Rank 1: 0,0009 (indica consistência no ajuste interno)  
   
   A combinação de gap reduzido e variância aceitável sinaliza um modelo **robusto** e **reprodutível**.

#### Escolha do Modelo

##### Fit

Vamos então selecionar o melhor modelo e dar fit nele com o dataset de treino completo.

In [ ]:
chosen_model = search.best_estimator_
chosen_model.fit(X_train, y_train_log)

Pipeline(memory='~/cachedir',
         steps=[('select_columns',
                 ColumnSelector(columns=['Garage.Type', 'MS.Zoning',
                                         'Bsmt.Qual', 'Central.Air',
                                         'Overall.Qual', 'Gr.Liv.Area',
                                         'Garage.Cars', 'Total.Bsmt.SF',
                                         'Garage.Area', 'Year.Built',
                                         'X1st.Flr.SF', 'Lot.Area',
                                         'BsmtFin.SF.1', 'Year.Remod.Add',
                                         'Overall.Cond'])),
                ('features',
                 FeatureUnion(transformer_list=[('base',
                                                 ColumnTr...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=3, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=300, n_jobs=None,
                              num_parallel_tree=None, ...))])

##### Previsão e Performance

Com o modelo "fitado", vamos prever o target do conjunto de testes e medir a sua performance.

In [ ]:
y_pred = chosen_model.predict(X_test)

y_value = np.expm1(y_pred)
rmse = np.sqrt(mean_squared_error(y_test, np.expm1(y_pred)))

print(rmse)

33279.92499991549


Para vizualizar a performance do modelo no conjunto de testes 

In [ ]:
true_vs_pred(y_test, y_value, 'true_vs_pred.png')

Gráfico salvo em ./graphs/true_vs_pred.png


Podemos observar neste scatterplot, que o nosso modelo acaba não lidando muito bem com valores muito altos ou valores muito baixos.

Para melhorar isso, é possível:

- Rever a remoção de outliers
- Rever a transformação com o log1p

## Etapa 3

### Deploy

Como vamos utilizar o modelo normalmente?

Para utilizar o modelo para prever de fácil aceso, vamos fazer uma api Flask, para isso, iniciamos treinando o modelo que escolhemos com o dataset inteiro, já que à partir de agora os dados serão novos.  
Depois, salvamos o modelo em um arquivo `.pkl`, para passar para um diretório onde a api irá rodar.

#### Fit

In [ ]:
chosen_model.fit(X, y)

_ = jbl.dump(chosen_model, 'model.pkl')

#### Data Example

Agora vamos criar um body de exemplo para passar para a api de previsão

In [ ]:
X_test_ = X_test.copy()

X_test_.reset_index(drop=True, inplace=True)

predicting_n = 1

example_house = dict(X_test_.iloc[predicting_n])
for k, v in example_house.items():
    if isinstance(v, np.int64):
        example_house[k] = int(v)
    elif isinstance(v, np.float64):
        example_house[k] = float(v)

example_house


{'MS.SubClass': 20,
 'MS.Zoning': 'RL',
 'Lot.Frontage': nan,
 'Lot.Area': 9156,
 'Street': 'Pave',
 'Alley': nan,
 'Lot.Shape': 'IR1',
 'Land.Contour': 'Lvl',
 'Utilities': 'AllPub',
 'Lot.Config': 'Inside',
 'Land.Slope': 'Gtl',
 'Neighborhood': 'NWAmes',
 'Condition.1': 'PosN',
 'Condition.2': 'Norm',
 'Bldg.Type': '1Fam',
 'House.Style': '1Story',
 'Overall.Qual': 6,
 'Overall.Cond': 7,
 'Year.Built': 1968,
 'Year.Remod.Add': 1968,
 'Roof.Style': 'Hip',
 'Roof.Matl': 'CompShg',
 'Exterior.1st': 'BrkFace',
 'Exterior.2nd': 'BrkFace',
 'Mas.Vnr.Type': nan,
 'Mas.Vnr.Area': 0.0,
 'Exter.Qual': 'TA',
 'Exter.Cond': 'TA',
 'Foundation': 'CBlock',
 'Bsmt.Qual': 'TA',
 'Bsmt.Cond': 'TA',
 'Bsmt.Exposure': 'No',
 'BsmtFin.Type.1': 'Unf',
 'BsmtFin.SF.1': 0.0,
 'BsmtFin.Type.2': 'Unf',
 'BsmtFin.SF.2': 0.0,
 'Bsmt.Unf.SF': 1489.0,
 'Total.Bsmt.SF': 1489.0,
 'Heating': 'GasA',
 'Heating.QC': 'Gd',
 'Central.Air': 'Y',
 'Electrical': 'SBrkr',
 'X1st.Flr.SF': 1489,
 'X2nd.Flr.SF': 0,
 'Low.Qua

#### Chamada da API

In [ ]:
# Chamando a api com o body dos dados

import requests
import json

url = "http://18.231.253.225:8080/api/v1/predict"
data = example_house

headers = {
    'Content-Type': 'application/json'
}

try:
    response = requests.post(url, data=json.dumps(data), headers=headers)
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")
    response = None


predicted_value = response.json()["predicted_value"]




#### Verificando o resultado